In [1]:
import DualwormFunctions as dw
import Observables as obs
%load_ext line_profiler
%load_ext memory_profiler
import RunBasis_NewSaves
import argparse
import StartStates as strst
import numpy as np

In [2]:
nst = 2;
nsm = 16;
nips = 2;
nrps = 4;
nb = 16;
measperiod = 2;
nt = 8
nh = 8

In [3]:
def genArgs():
    parser = argparse.ArgumentParser()
    
    parser.add_argument('-f', type = str)

    parser.add_argument('--L', type = int, default = 12, help = 'Lattice side size')

    # COUPLINGS
    parser.add_argument('--J1', type = float, default = 1,
                        help = 'NN coupling') # nearest-neighbour coupling
    parser.add_argument('--J2', type = float, default = 0,
                        help = '2nd NN coupling') # 2nd NN coupling
    parser.add_argument('--J3', type = float, default = 0,
                        help = '3rd NN coupling') # 3rd NN coupling
    parser.add_argument('--J4', type = float, default = 0.0,
                        help = '4th NN coupling')
    parser.add_argument('--h', type = float, default = 1.0,
                        help = 'Magnetic field')    
    #NUMBER OF STEPS AND ITERATIONS
    parser.add_argument('--nst', type = int, default = nst,
                        help = 'number of thermalisation steps') # number of thermalisation steps
    parser.add_argument('--nsm', type = int, default = nsm,
                        help = 'number of measurements steps') # number of measurement steps
    parser.add_argument('--nips', type = int, default = nips,
                        help = 'number of worm constructions per MC step')
    parser.add_argument('--nrps', type = int, default = nrps,
                        help = 'number of replica loops per MC step')
    parser.add_argument('--measperiod', type = int, default = measperiod,
                        help = 'number of nips worm building + swaps between measurements')
    parser.add_argument('--nb', type = int, default = nb,
                        help = 'number of bins')

    #PARALLELISATION
    parser.add_argument('--ncores', type = int, default = 6,
                        help = 'number of threads to use')

    
    # INITIALISATION PARAMETERS
    parser.add_argument('--randominit', default = True, action ='store_true',
                        help = 'intialise the states randomly')
    parser.add_argument('--same', default = False, action = 'store_true',
                        help = '''initialise all temperatures with the same
                        state (debug purposes)''')
    parser.add_argument('--magninit', default = False, action = 'store_true',
                        help = '''initialise all the temperature with
                        one of the m=1/3 GS''')
    parser.add_argument('--magnstripes', default = False, action = 'store_true',
                       help = '''initialise all the temperature with
                       m=1/3 stripes''')
    parser.add_argument('--maxflip', default = False, action = 'store_true',
                       help = '''initialise all the temperature with
                       maximally flippable plateau''')
    parser.add_argument('--loadfromfile', default = False, action = 'store_true',
                       help = '''initialise all the states with
                       results from a previously performed simulations''')
    parser.add_argument('--filename', type = str, default = 'tests/J1hLongRun_RndInit_AlternateUpdateResults',
                       help = '''initialise all the states with
                       results from a previously performed simulations''')
    
    #WORM PARAMETERS
    parser.add_argument('--nmaxiter', type = int, default = 10,
                        help = '''maximal number of segments in a loop update over the
                        size of the lattice (1 = 1times the number of dualbonds in the
                        lattice)''')
    parser.add_argument('--measupdate', default = False, action = 'store_true',
                       help = '''activate to mimic the action of the measuring tip''')
    parser.add_argument('--p', type = float, default = 0.0, 
                       help = '''prob of the measuring tip flipping the spin (number between 0 and 1)''')
    parser.add_argument('--ssf', default = True, action = 'store_true',
                        help = 'activate for single spin flip update')
    parser.add_argument('--alternate', default = True, action = 'store_true',
                        help = 'activate for single spin flip update and dw update')
    parser.add_argument('--checkgs', default = True, action = 'store_true',
                        help = 'activate to debug ssf')
    
    #TEMPERATURE PARAMETERS
    parser.add_argument('--t_list', nargs = '+', type = float, default = [0.05,60],
                        help = 'list of limiting temperature values')
    parser.add_argument('--nt_list', nargs = '+', type = int, default = [nt],
                        help = 'list of number of temperatures in between the given limiting temperatures')
    parser.add_argument('--log_tlist', default = False, action='store_true',
                        help = 'state whether you want the temperature be spaced log-like or linear-like (activate if you want log)')
    parser.add_argument('--stat_temps_lims', nargs = '+', type = float,
                        help = '''limiting temperatures for the various ranges of
                        measurements''') 
                        #default will be set to none, and then we can decide what to do later on.
    #MAGNETIC FIELD PARAMETERS
    parser.add_argument('--h_list', nargs = '+', default = [0.2,6.0],type = float,
                        help = 'list of limiting magnetic field values')
    parser.add_argument('--nh_list', nargs = '+', default = [nh], type = int,
                        help = 'list of number of magnetic fields in between the given limiting temperatures')
    parser.add_argument('--stat_hfields_lims', nargs = '+', type = float,
                help = '''limiting magnetic fields for the various ranges of
                measurements''') 
    #CORRELATIONS PARAMETER
    parser.add_argument('--energy', default = True, action = 'store_true',
                        help = 'activate if you want to save the energy')
    parser.add_argument('--magnetisation', default =True, action = 'store_true',
                        help = 'activate if you want to save the magnetisation')
    parser.add_argument('--charges', default = False, action = 'store_true',
                        help = 'activate if you want to save the charges')
    parser.add_argument('--correlations', default = True, action = 'store_true',
                        help = 'activate if you want to save either central or all correlations')
    parser.add_argument('--all_correlations', default = False, action = 'store_true',
                        help = '''activate if you want to save the correlations for all non-equivalent
                        pairs of sites. Otherwise, will save central correlations.''')
    parser.add_argument('--firstcorrelations', default = False, action = 'store_true',
                        help = 'activate if you want to save first correlations, otherwise will save central')
    #SAVE
    parser.add_argument('--output', type = str, default = "tests", help = 'saving filename (.pkl will be added)')
    args = parser.parse_args()
    return args

In [4]:
def callRunBasis():
    args = genArgs()
    return RunBasis_NewSaves.main(args)

In [5]:
%lprun -f dw.mcs_swaps meanstat, swapst, swapsh, failedupdatesth, failedupdates = callRunBasis() 

-------------------Initialisation--------------------
Lattice side size:  12
J1  1
J2  0
J3  0
J3st  0
Couplings extracted
Hamiltonian expression (without field) computed
single spin flip update
alternating ssf and dw update
Number of temperatures:  8
Temperatures: [5.00000000e-02 8.61428571e+00 1.71785714e+01 2.57428571e+01
 3.43071429e+01 4.28714286e+01 5.14357143e+01 6.00000000e+01]
Number of magnetic fields:  8
Magnetic fields:  [0.2        1.02857143 1.85714286 2.68571429 3.51428571 4.34285714
 5.17142857 6.        ]
Fully random initialisation =  True
Identical initialisation =  False
Magnetisation initialisation =  False
[ -950.4        -1308.34285714 -1666.28571429 -2024.22857143
 -2382.17142857 -3036.34285714 -4110.17142857 -5184.        ]
RunBasis: Away from gs at t index  0  and h index  0
   new_en_states[t,h] =  80.8
   ref_en_states[h] =  -950.4
RunBasis: Away from gs at t index  0  and h index  1
   new_en_states[t,h] =  93.6
   ref_en_states[h] =  -1308.3428571428572
Ru

In [6]:
print(failedupdatesth/(nst*nips))

[[0.96875    0.96643519 0.9220679  0.9068287  0.91859568 0.8751929
  0.88734568 0.88869599]
 [0.15393519 0.19077932 0.2038966  0.24517747 0.28780864 0.31577932
  0.38599537 0.44193673]
 [0.08777006 0.10358796 0.11670525 0.14216821 0.16859568 0.20524691
  0.24440586 0.27295525]
 [0.05748457 0.06770833 0.07986111 0.10127315 0.12692901 0.14332562
  0.16415895 0.19077932]
 [0.04282407 0.05054012 0.06153549 0.07908951 0.09375    0.1095679
  0.13869599 0.1560571 ]
 [0.03607253 0.04861111 0.04378858 0.06346451 0.07908951 0.09008488
  0.11342593 0.12866512]
 [0.02893519 0.0320216  0.04340278 0.05189043 0.0628858  0.07600309
  0.09529321 0.1099537 ]
 [0.02449846 0.02584877 0.03317901 0.04494599 0.05767747 0.06539352
  0.07908951 0.09201389]]


In [7]:
print(failedupdates/(nsm*measperiod*nips))

[[0.99996383 1.00036169 0.99940924 1.00036169 1.00030141 1.00022907
  1.00024113 1.00025318]
 [0.16752267 0.19386574 0.20308883 0.2396195  0.28326341 0.32520737
  0.38648968 0.44070698]
 [0.08981964 0.10472126 0.11277488 0.13992573 0.16839072 0.20163002
  0.23913725 0.27471547]
 [0.06046248 0.07085503 0.07913773 0.09862076 0.12015336 0.14574894
  0.17062114 0.20059317]
 [0.0455488  0.05251736 0.06082417 0.07636478 0.09448544 0.11317274
  0.13492236 0.15489969]
 [0.036386   0.04246238 0.04920187 0.06253617 0.07632861 0.09276138
  0.10994165 0.12713397]
 [0.02985147 0.03575907 0.04106385 0.0522039  0.06463397 0.07716049
  0.09338831 0.10745804]
 [0.02523389 0.03135851 0.03524064 0.04603106 0.05571229 0.06961323
  0.08110291 0.09352093]]


In [8]:
print(4*swapst/(nrps*nsm*measperiod*nh))

[0.         0.0078125  0.203125   0.52734375 0.63671875 0.71875
 0.8359375  0.        ]


In [9]:
print(4*swapsh/(nrps*nsm*measperiod*nt))

[0.1875     0.46875    0.4140625  0.42578125 0.40625    0.46875
 0.56640625 0.3046875 ]
